각 파일에서 어떤 데이터가 저장되어 있고 어떤 데이터 필드로 구성되어 있는지 파악하라
pandas로 데이터를 확인하라
사용자 수, 영화 수, 평점 수, 제작국가 수, 출연진 수, 장르 개수 등의 기본 통계를 확인하라.

In [1]:
import pandas as pd

file_names = [
    'castings.csv',
    'countries.csv',
    'genres.csv',
    'rates.csv',
    'movies.txt',
    'peoples.txt',
]

names = [file_name.split('.')[0] for file_name in file_names]

data_list = dict()

def get_dataset_from_file(path: str, sep: str = ','):
  extension = path.split('.')[-1]
  if extension == "csv":
    return pd.read_csv(path, sep=sep, on_bad_lines='skip')
  elif extension == "txt":
    return pd.read_csv(path, sep='\t', on_bad_lines='skip')

for idx, file_name in enumerate(file_names):
  file_path = f"./kmrd-small/{file_name}"
  name = names[idx]
  data_list[name] = get_dataset_from_file(file_path)
  print(f"\n\n--- {name} ---\n")
  print(data_list[name].info())
  print(f"데이터 필드:\n{data_list[name].columns.tolist()}")
  print(f"데이터 구성:\n{data_list[name].head(1)}")
  print(f"데이터 수: {len(data_list[name])}")



--- castings ---

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9776 entries, 0 to 9775
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   movie    9776 non-null   int64
 1   people   9776 non-null   int64
 2   order    9776 non-null   int64
 3   leading  9776 non-null   int64
dtypes: int64(4)
memory usage: 305.6 KB
None
데이터 필드:
['movie', 'people', 'order', 'leading']
데이터 구성:
   movie  people  order  leading
0  10001    4374      1        1
데이터 수: 9776


--- countries ---

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1109 entries, 0 to 1108
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movie    1109 non-null   int64 
 1   country  1109 non-null   object
dtypes: int64(1), object(1)
memory usage: 17.5+ KB
None
데이터 필드:
['movie', 'country']
데이터 구성:
   movie country
0  10001    이탈리아
데이터 수: 1109


--- genres ---

<class 'pandas.core.frame.DataFrame'>
RangeInde

In [2]:
import re

def split_and_eleminate_by_char(item_set: set, seperator: str) -> set:
    result = set()
    pattern = f"[{re.escape(seperator)}]+"
    for item in item_set:
        list_in_item = re.split(pattern, item)
        result.update(set(list_in_item))
    return result

In [3]:
print(f"사용자 수: {len(set(data_list['rates']['user']))}")
print(f"영화 수: {len(set(data_list['movies']['movie']))}")
print(f"평점 수: {len(data_list['rates'])}")
print(f"제작국가 수: {len(set(data_list['countries']['country']))}")
print(f"출연진 수: {len(set(data_list['peoples']['people']))}")
print(f"장르 수: {len(pd.Series(data_list['genres']['genre'].str.split('/').sum()).unique())}\n\n")

print(f"사용자 수: {len(data_list['rates']['user'].unique())}")
print(f"영화 수: {len(data_list['movies']['movie'].unique())}")
print(f"평점 수: {len(data_list['rates'])}")
print(f"제작국가 수: {len(data_list['countries']['country'].unique())}")
print(f"출연진 수: {len(data_list['peoples']['people'].unique())}")
print(f"장르 수: {len(split_and_eleminate_by_char(set(data_list['genres']['genre']), '/'))}")


사용자 수: 52028
영화 수: 999
평점 수: 140710
제작국가 수: 36
출연진 수: 7172
장르 수: 21


사용자 수: 52028
영화 수: 999
평점 수: 140710
제작국가 수: 36
출연진 수: 7172
장르 수: 21


- 데이터에 결측치와 이상치가 있는지 확인, 처리
- 사용자별/영화별 평점 분포를 히스토그램, 박스 플롯으로 시각화하기

#### 결측치와 이상치
> 결측치
- 데이터셋에서 특정 값이 누락된 것을 의미
- 수집오류, 입력 실수, 의도적 누락, 병합 오류..
- 종류
  - MCAR(Missing Completely at Random)
    결측치 발생이 완전히 무작위적이며 다른 변수나 값과 관계 없음
  - MAR(Missing at Random)
    결측치 발생이 특정 변수와는 관련 있지만 관측되지 않은 값과는 무관함
  - MNAR(Missing Not at Random)
    결측치 발생이 데이터 자체의 특성과 관련됨.

> 이상치
- 비상식적으로 크거나 작은 값. 동떨어진 값.
- 입력, 시스템 오류, 자연스러운 변동
- 종류
  - 극단치, 고립된 이상치, Multivariate 이상치
- 기술 통계 확인(IRQ)



In [4]:
# 결측치 제거
data_list_dropna = dict()
for name in names:
  data_list_dropna[name] = data_list[name].dropna()

# 결측치 채우기
data_list_fillna = dict()
for name in names:
  data_list_fillna[name] = data_list[name].fillna(0)

# 이상치 제거
df = data_list['rates']
Q1 = df['rate'].quantile(0.25)
Q3 = df['rate'].quantile(0.75)
IQR = Q3 - Q1

print(Q1, Q3, IQR)


lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

print(lower_bound, upper_bound)
outliers = df[(df['rate'] >= lower_bound) & (df['rate'] <= upper_bound)]
print(f"rates len: {len(df)}, outlier len: {len(outliers)}")

9.0 10.0 1.0
7.5 11.5
rates len: 140710, outlier len: 120609


In [5]:

print(f"사용자 수: {len(data_list_dropna['rates']['user'].unique())}")
print(f"영화 수: {len(data_list_dropna['movies']['movie'].unique())}")
print(f"평점 수: {len(data_list_dropna['rates'])}")
print(f"제작국가 수: {len(data_list_dropna['countries']['country'].unique())}")
print(f"출연진 수: {len(data_list_dropna['peoples']['people'].unique())}")
print(f"장르 수: {len(split_and_eleminate_by_char(set(data_list_dropna['genres']['genre']), '/'))}\n\n")


print(f"사용자 수: {len(data_list['rates']['user'].unique())}")
print(f"영화 수: {len(data_list['movies']['movie'].unique())}")
print(f"평점 수: {len(data_list['rates'])}")
print(f"제작국가 수: {len(data_list['countries']['country'].unique())}")
print(f"출연진 수: {len(data_list['peoples']['people'].unique())}")
print(f"장르 수: {len(split_and_eleminate_by_char(set(data_list['genres']['genre']), '/'))}")

사용자 수: 52028
영화 수: 599
평점 수: 140710
제작국가 수: 36
출연진 수: 6305
장르 수: 21


사용자 수: 52028
영화 수: 999
평점 수: 140710
제작국가 수: 36
출연진 수: 7172
장르 수: 21
